In [2]:
import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML
#from query_representation.utils import *

#from moz_sql_parser import parse
#from mo_sql_parsing import parse as parse
import os
import pandas as pd
from collections import defaultdict

from IPython.display import HTML
from IPython.display import display_html

In [3]:
#QDIR = "./queries/tpcds/all"
#QDIR = "queries/job/all_job/sqls/all_job"
#QDIR = "ceb-unique-sqls"

#QDIR = "../scraping-queries/sqls"
QDIR = "../data/job/all_job/sqls"

TPCDS_TABLES="./queries/tpcds/tpcds.sql"
COLS_TO_TABLES = {}


In [4]:

if "tpcds" in QDIR:
    with open(TPCDS_TABLES) as f:
        tdata = f.readlines()

    for li, line in enumerate(tdata):
        #print(line)
        if "create table" in line:
            #print(line)
            vals = line.split(" ")
            #print("table: ", vals[2])
            table = vals[2].replace("\n", "")

            for i in range(1000):
                cline = tdata[li+i]
                cline = cline.lstrip()
                #print(cline)

                if "create table" in cline or "primary key" in cline:
                    continue
                vals = cline.split(" ")
                col = vals[0]

                if not (")" in col or "(" in col or col == ""):
                    if col in COLS_TO_TABLES:
                        print("?>>???")
                        print(col)
                    assert col not in COLS_TO_TABLES   
                    COLS_TO_TABLES[col] = table

                if ")" in cline and "," not in cline:
                    break
        

In [5]:
#print(COLS_TO_TABLES)

In [6]:
#"catery" in COLS_TO_TABLES
for k in COLS_TO_TABLES:
    if "cat" in k:
        print(k)

In [7]:
FNS = os.listdir(QDIR)

In [8]:
print(len(FNS))
print(FNS[19])

113
6a.sql


In [16]:
sqls = []
for fn in FNS:
    if ".sql" not in fn:
        continue
    if "all" in fn:
        continue
    fn = os.path.join(QDIR, fn)
    with open(fn, "r") as f:
        sql = f.read()
    #if "like" in sql.lower():
        #print(sql)
    sql = sql.replace("IN", "in")
    sql = sql.replace("In", "in")
    sql = sql.replace("ILIKE", "like")
    sql = sql.replace("LIKE", "like")
    sqls.append(sql)

In [17]:
# import json
# with open("../db1_queries.json", "r") as f:
#     data = json.loads(f.read())
# sqls = [data[k]["sql"] for k in data]

In [18]:
for i, sql in enumerate(sqls):
    sql = sql.replace("IN", "in")
    sql = sql.replace("In", "in")
    sql = sql.replace("ILIKE", "like")
    sql = sql.replace("LIKE", "like")
    sqls[i] = sql

In [19]:
print(len(sqls))

171352


In [20]:
# for si, sql in enumerate(sqls):
#     if "like" in sql.lower():
#         print(si)
#     if "q91" in sql.lower():
#         print(si)

In [21]:
#sql = sqls[40]
sql = sqls[78]
# sql = sql.replace("IN", "in")
# sql = sql.replace("In", "in")
# sql = sql.replace("ILIKE", "like")

In [22]:
print(sql)

-- Enter Query Title
-- Enter Query Description

sp_help badges


In [23]:
# def get_cols_from_tokens(tokens):
#     cols = []
#     for token in tokens:
#         if isinstance(token, sqlparse.sql.Identifier):
#             cols.append(str(token.value))
#     return cols

def get_function_cols(func_token):
    cols = []
    for param in func_token.get_parameters():
        if isinstance(param, sqlparse.sql.Identifier):
            cols.append(param.value)
    return cols

def parse_sql_preds(sql):
    def get_table_col(left):
        if "." in left:
            tablename = left[0:left.find(".")]
            col = left[left.find(".")+1:]
        else:
            tablename = "X"
            col = left
        return tablename, col
        
    def parse_where(wtokens):
        curwhere = {}
        for idx, token in enumerate(wtokens):
            if "where" in str(type(token)).lower():
                parse_where(token)
                continue
            
            if isinstance(token, sqlparse.sql.Parenthesis):
                assert hasattr(token, "tokens")
                #parse_recursive(token)
                parse_where(token)
                continue
            
            if "comparison" in str(type(token)).lower():
                #print(token.value)
                assert hasattr(token, "tokens")
                left = None
                op = None
                right = None
                valtypes = []
        
                for ctoken in token:
                    if ctoken.value.strip() == "":
                        continue
                    assert right is None
            
                    if left is None:
                        left = ctoken
                        valtypes.append(type(ctoken))
                        continue
                    if op is None:
                        op = ctoken
                        continue
                        
                    if right is None:
                        right = ctoken
                        continue
                    
                if isinstance(left, sqlparse.sql.Identifier):
                    #left = left.value
                    tablename, col = get_table_col(left.value)  
                elif isinstance(left, sqlparse.sql.Function):
                    tablename = "function"
                    allcols = get_function_cols(left)
                    #print("Num function columns: ", len(allcols))
                    col = ",".join(allcols)
                elif isinstance(left, sqlparse.sql.Parenthesis):
                    assert hasattr(left, "tokens")
                    parse_where(left)
                    tablename = "function"
                    col = "function"
                elif isinstance(left, sqlparse.sql.Operation):
                    tablename = "function"
                    col = "function"
                else:
                    # FIXME: handle these better
                    if isinstance(right, sqlparse.sql.Identifier):
                        tablename, col = get_table_col(right.value)
                        tmp1 = right
                        right = left
                        left = tmp1
                        # FIXME: need to change other stuff, like operator type too
                    else:
                        assert False
                    
                if "catery" in col:
                    col = col.replace("catery", "category")
                    
                if isinstance(right, sqlparse.sql.Identifier):
                    #print("Join?")
                    continue
                
                if isinstance(right, sqlparse.sql.Parenthesis):
                    assert hasattr(right, "tokens")
                    parse_recursive(right)
                
                # TODO: need a better way to find joins
                if not "token" in str(type(right)).lower():
                    if not "select" in right.value and \
                        op.value == "=" and \
                        "." in right.value:
                        # potential join
                        continue
                
                op = op.value.lower()
                right = right.value
                op = op.lower()
                
                allpreddata["left"].append(left.value)
                allpreddata["table"].append(tablename)   
                allpreddata["col"].append(col)
                allpreddata["op"].append(op)
                allpreddata["val"].append(str(right))
            
            #### TODO:
            if "between" in str(token):
                left = wtokens[idx-2]
                
                if isinstance(left, sqlparse.sql.Identifier):
                    tablename, col = get_table_col(left.value)
                    #print(tablename, col)
                elif isinstance(left, sqlparse.sql.Parenthesis):
                    tablename = "betweenX"
                    col = "func"
                else:
                    tablename = "betweenX"
                    col = "unk"
                    #assert False
                
                allpreddata["left"].append(left.value)
                allpreddata["table"].append(tablename)
                allpreddata["col"].append(col)
                allpreddata["op"].append("between")
                allpreddata["val"].append(-1)
                
#                 left = wtokens[idx+2]
#                 right = wtokens[idx+6]
# FIXME: general alternative
#                 for ci in range(-2,7,1):
#                     print(ci)
#                     print(wtokens[idx+ci])
            
    def parse_recursive(tokens):
        for idx, token in enumerate(tokens):
            if hasattr(token, "tokens"):
                parse_recursive(token)
            
            # original code
            if "where" in str(type(token)).lower():
                parse_where(token)
            
            if token.value.lower() in ["having", "group by"]:
                # TODO: parse
                index, token2 = tokens.token_next(idx)
        
    allpreddata = defaultdict(list)
    allpreds = []
    
    if sql.strip() == "":
        return pd.DataFrame(allpreddata)

    parsed = sqlparse.parse(sql)
    assert len(parsed) == 1
    parse_recursive(parsed[0])
    #return allpreds
    df = pd.DataFrame(allpreddata)
    return df

In [24]:
#print(sql)

In [25]:
preds = parse_sql_preds(sql)
preds

""


In [26]:
# #print(preds)
# p0 = preds[0]

# mergedd = {}
# for p in preds:
#     for table,colvals in p.items():
#         if table not in mergedd:
#             mergedd[table] = {}
#         for col,opvals in colvals.items():
#             if col not in mergedd[table]:
#                 mergedd[table][col] = {}
#             for op,vals in opvals.items():
#                 if op not in mergedd[table][col]:
#                     mergedd[table][col][op] = set()
#                 for cval in vals:
#                     mergedd[table][col][op].add(cval)
                

# #print(mergedd)
# for table,vals in mergedd.items():
#     print(table)
#     print(vals)
#     print("***********")

# Q87 very hard to parse;
### TODO: maybe queries w/ multiple subqueries, we just try to get each select block and parse them individually

In [27]:
# sql4 = """-- q87


# select count(*) 
# from (select distinct c_last_name, c_first_name, d_date
#        from store_sales, date_dim, customer  -- skan_memo_stash_87
#        where store_sales.ss_sold_date_sk = date_dim.d_date_sk
#          and store_sales.ss_customer_sk = customer.c_customer_sk
#          and d_month_seq between 1212 and 1212+11)
#        except
#       (select distinct c_last_name, c_first_name, d_date
#        from catalog_sales, date_dim, customer
#        where catalog_sales.cs_sold_date_sk = date_dim.d_date_sk
#          and catalog_sales.cs_bill_customer_sk = customer.c_customer_sk
#          and d_month_seq between 1212 and 1212+11)
#        except
#       (select distinct c_last_name, c_first_name, d_date
#        from web_sales, date_dim, customer
#        where web_sales.ws_sold_date_sk = date_dim.d_date_sk
#          and web_sales.ws_bill_customer_sk = customer.c_customer_sk
#          and d_month_seq between 1212 and 1212+11)
# cool_cust

# """

# # sql4 = """-- q87


# # select count(*) 
# # from ((select distinct c_last_name, c_first_name, d_date
# #        from store_sales, date_dim, customer  -- skan_memo_stash_87
# #        where store_sales.ss_sold_date_sk = date_dim.d_date_sk
# #          and store_sales.ss_customer_sk = customer.c_customer_sk
# #          and d_month_seq between 1212 and 1212+11)
# #        except
# #       (select distinct c_last_name, c_first_name, d_date
# #        from catalog_sales, date_dim, customer
# #        where catalog_sales.cs_sold_date_sk = date_dim.d_date_sk
# #          and catalog_sales.cs_bill_customer_sk = customer.c_customer_sk
# #          and d_month_seq between 1212 and 1212+11))
# # cool_cust

# # """

# # sql4 = """-- q87


# # select count(*) 
# # from (select distinct c_last_name, c_first_name, d_date
# #        from store_sales, date_dim, customer  -- skan_memo_stash_87
# #        where store_sales.ss_sold_date_sk = date_dim.d_date_sk
# #          and store_sales.ss_customer_sk = customer.c_customer_sk
# #          and d_month_seq between 1212 and 1212+11)
# # cool_cust
# # """
# parse_sql_preds(sql4)

# Fix for Q40, change order of from tables

In [28]:

sql3 = """select top 100 
   w_state
  ,i_item_id
  ,sum(case when (cast(d_date as date) < cast ('1998-04-08' as date)) 
        then cs_sales_price - coalesce(cr_refunded_cash,0) else 0 end) as sales_before
  ,sum(case when (cast(d_date as date) >= cast ('1998-04-08' as date)) 
        then cs_sales_price - coalesce(cr_refunded_cash,0) else 0 end) as sales_after
 from
   warehouse  -- skan_memo_stash_40
  ,item
  ,date_dim
  ,catalog_sales left outer join catalog_returns on
       (cs_order_number = cr_order_number 
        and cs_item_sk = cr_item_sk)

 where
     i_current_price between 0.99 and 1.49
 and i_item_sk          = cs_item_sk
 and cs_warehouse_sk    = w_warehouse_sk 
 and cs_sold_date_sk    = d_date_sk
 and d_date >= dateadd(dd, -30, '1998-04-08')
 and d_date <= dateadd(dd, 30, '1998-04-08') 
 group by
    w_state,i_item_id
 order by w_state,i_item_id"""
parse_sql_preds(sql3)

,left,table,col,op,val
0,i_current_price,X,i_current_price,between,-1
1,d_date,X,d_date,>=,"dateadd(dd, -30, '1998-04-08')"
2,d_date,X,d_date,<=,"dateadd(dd, 30, '1998-04-08')"


# replace intersect by union seems to work for now;

In [29]:
sql2 = """select top 100 count(*) from (
    select distinct c_last_name, c_first_name, d_date
    from store_sales, date_dim, customer -- skan_memo_stash_38
          where store_sales.ss_sold_date_sk = date_dim.d_date_sk
      and store_sales.ss_customer_sk = customer.c_customer_sk
      and d_month_seq between 1212 and 1212 + 11) test"""
sql2 = """select top 100 count(*) from (
    select distinct c_last_name, c_first_name, d_date
    from store_sales, date_dim, customer -- skan_memo_stash_38
          where store_sales.ss_sold_date_sk = date_dim.d_date_sk
      and store_sales.ss_customer_sk = customer.c_customer_sk
      and d_month_seq between 1212 and 1212 + 11
  union
    select distinct c_last_name, c_first_name, d_date
    from catalog_sales, date_dim, customer
          where catalog_sales.cs_sold_date_sk = date_dim.d_date_sk
      and catalog_sales.cs_bill_customer_sk = customer.c_customer_sk
      and d_month_seq between 1212 and 1212 + 11) tmp"""
parse_sql_preds(sql2)

,left,table,col,op,val
0,d_month_seq,X,d_month_seq,between,-1
1,d_month_seq,X,d_month_seq,between,-1


In [30]:
def is_num(val):
    try:
        float(val)
        return True
    except:
        return False
    
def is_a_date(val):
    
    val = val.replace("'", "")
    if "date" in val:
        return True
    
    elif is_num(val):
        num = float(val)
        if num > 1900 and num < 2010:
            return True
        
    elif "+" in val:
        cvals = val[0:val.find("+")]
        if is_num(cvals):
            num = float(cvals)
            if num > 1900 and num < 2010:
                return True
            
    elif "-" in val:
#         if val.count("-") == 2:
#             print(val)
        cvals = val[0:val.find("-")]
        if is_num(cvals):
            num = float(cvals)
            if num > 1900 and num < 2010:
                return True
    
    return False


#import enchant
#enchantD = enchant.Dict("en_US")
import re
#import nltk
#nltk.download('words')
#from nltk.corpus import words as nltkwords

URLPAT = "((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*"
def get_like_type(op, vals):
    likekind = "-1"
    likedtype = "-1"
    
    if "like" not in op.lower():
        return likedtype, likekind
    
    vals = vals.replace("'", "")
    vals = vals.replace("(", "")
    vals = vals.replace(")", "")
    
    if vals.count("%") == 0:
        likekind = "no%"
    elif vals.count("%") == 2:
        if vals[0] == "%" and vals[-1] == "%":
            likekind = "contains"
        else:
            likekind = "x%y%x"
    elif vals.count("%") == 1:
        if vals[0] == "%":
            likekind = "ends"
        elif vals[-1] == "%":
            likekind = "starts"
        else:
            likekind = "x%x"
    else:
        nlikes = vals.count("%")
        likekind = str(nlikes) + "%"
    
    opval = vals.replace("%", "")
    
    if is_num(opval):
        likedtype = "num"
    elif len(opval) <= 2:
        likedtype = "short"
    elif opval[0] == ".":
        likedtype = "extension"
    elif re.match(URLPAT, opval) is not None:
        if ("cosmos" in opval or "adl" in opval) and "/" in opval:
            likedtype = "path"
        else:
            likedtype = "url"
            
    elif opval.count("/") >= 2 or opval.count("\\") >= 2:
        likedtype = "path"
        
    #elif enchantD.check(opval):
    elif False:
        likedtype = "word"

    elif opval.count("-") >= 2 or \
            opval.count(":") >= 2:
        likedtype = "serial"
    elif "-" in opval or "_" in opval or " " in opval or "," in opval or ":" in opval:
        if "-" in opval:       
            allvals = opval.split("-")
        elif "_" in opval:
            allvals = opval.split("_")
        elif " " in opval:
            allvals = opval.split(" ")
        elif "," in opval:
            allvals = opval.split(",")
        elif ":" in opval:
            allvals = opval.split(":")
            
        validwords = 0
        for v1 in allvals:
            if v1 == "":
                continue
            #if enchantD.check(v1):
            if False:
                validwords += 1
        if validwords >= 2:
            likedtype = "words"
    elif "0x" in opval:
        likedtype = "hex"
    else:
        validwords = 0
        start = 0
        prevstart = 0
        for oi,_ in enumerate(opval):
            cword = opval[start:oi+1]
            pword = opval[prevstart:oi+1]
            #if enchantD.check(cword) and len(cword) >= 3:
            if False:
                validwords += 1
                prevstart = start
                start = oi+1
            
            # extend previous word
            #elif enchantD.check(pword) and len(pword) >= 3:
            elif False:
                start = oi+1
                
        if validwords >= 2:
            likedtype = "words"
        else:
            likedtype = "unknown"
    
    return likedtype, likekind

def get_discrete_type(op, vals):
    dtype = "-1"
    dkind = "-1"
    #op = op.lower()
    assert op != ""
    if op in ["=", "!=", "<>", "in"]:
        if op == "=":
            dkind = "eq"
        elif op in ["!=", "<>"]:
            #print("neq!!")
            dkind = "neq"
        else:
            dkind = "in"
            
        if "select" in vals:
            dtype = "sql"
        else: 
            if "(" in vals:
                vals = vals.replace("(", "")
                vals = vals.replace(")", "")
                vals = vals.split(",")
                vals = vals[0]
            
            if is_a_date(vals):
                dtype = "date"
            elif is_num(vals):
                dtype = "num"
            else:
                dtype = "string"  
        
    return dtype, dkind

In [34]:
from collections import defaultdict

fails = 0
allinpdata = defaultdict(list)
allpreds = []

for si, sql in enumerate(sqls):
    # empty sqls
#     if si in [78, 85]:
#         continue
    
    if si % 100 == 0:
        print("Query: ", si)
        
    alldfdata = defaultdict(list)
    numwheres = sql.lower().count("where")
    
    try:
        preds = parse_sql_preds(sql)
    except Exception as e:
        #print("FAIL")
        fails += 1
        continue
    
    olen = len(preds)
    preds = preds.drop_duplicates()
    nlen = len(preds)
    
    preds["qid"] = si
    
    curcols = []
    curops = []
    for idx, row in preds.iterrows():
        curcols.append(row["col"])
        curops.append(row["op"])
        
        if row["col"] in COLS_TO_TABLES:
            tab = COLS_TO_TABLES[row["col"]]
            col = row["col"]
        elif row["table"] != "X":
            tab = row["table"]
            col = row["col"]
        else:
            tab = row["table"]
            col = "X"
        
        op = row["op"]
        alldfdata["table"].append(tab)
        alldfdata["col"].append(col)
        dtype,dkind = get_discrete_type(op, row["val"])
        alldfdata["dtype"].append(dtype)
        alldfdata["dkind"].append(dkind)
        
        likedtype,likekind = get_like_type(op, row["val"])
        alldfdata["likedtype"].append(likedtype)
        alldfdata["likekind"].append(likekind)
        
        if op in ["=", "!="]:
            alldfdata["discrete_ops"].append(1)
            alldfdata["in_ops"].append(0)
            allinpdata["col"].append(row["col"])
            allinpdata["val"].append(row["val"])
            
        elif op == "in":
            alldfdata["discrete_ops"].append(1)
            alldfdata["in_ops"].append(1)
            if "select" in row["val"]:
                pass
            else:
                cvals = row["val"]
                cvals = cvals.replace("(", "")
                cvals = cvals.replace(")", "")
                cvals = cvals.split(",")
                for cval in cvals:
                    allinpdata["col"].append(row["col"])
                    allinpdata["val"].append(cval)
        else:
            alldfdata["discrete_ops"].append(0)
            alldfdata["in_ops"].append(0)

        if op in [">", "<", ">=", "<=", "between"]:
            alldfdata["cont_ops"].append(1)
        else:
            alldfdata["cont_ops"].append(0)

        if op == "like":
            alldfdata["like_ops"].append(1)
        else:
            alldfdata["like_ops"].append(0)
    
    if len(preds) == 0:
        continue
    
    tmpdf = pd.DataFrame(alldfdata)
    assert len(tmpdf) == len(preds)
#     preds["like_ops"] = tmpdf["like_ops"]
#     preds["cont_ops"] = tmpdf["cont_ops"]
#     preds["discrete_ops"] = tmpdf["discrete_ops"]
#     preds["in_ops"] = tmpdf["in_ops"]
#     preds["dtype"] = tmpdf["dtype"]
#     preds["dkind"] = tmpdf["dkind"]
#     preds["likekind"] = tmpdf["likekind"]
    for k in tmpdf.keys():
        preds[k] = tmpdf[k]
        
    allpreds.append(preds)

Query:  0
Query:  100
Query:  200
Query:  300
Query:  400
Query:  500
Query:  600
Query:  700
Query:  800
Query:  900
Query:  1000
Query:  1100
Query:  1200
Query:  1300
Query:  1400
Query:  1500
Query:  1600
Query:  1700
Query:  1800
Query:  1900
Query:  2000
Query:  2100
Query:  2200
Query:  2300
Query:  2400
Query:  2500
Query:  2600
Query:  2700
Query:  2800
Query:  2900
Query:  3000
Query:  3100
Query:  3200
Query:  3300
Query:  3400
Query:  3500
Query:  3600
Query:  3700
Query:  3800
Query:  3900
Query:  4000
Query:  4100
Query:  4200
Query:  4300
Query:  4400
Query:  4500
Query:  4600
Query:  4700
Query:  4800
Query:  4900
Query:  5000
Query:  5100
Query:  5200
Query:  5300
Query:  5400
Query:  5500
Query:  5600
Query:  5700
Query:  5800
Query:  5900
Query:  6000
Query:  6100
Query:  6200
Query:  6300
Query:  6400
Query:  6500
Query:  6600
Query:  6700
Query:  6800
Query:  6900
Query:  7000
Query:  7100
Query:  7200
Query:  7300
Query:  7400
Query:  7500
Query:  7600
Query:  770

Query:  59400
Query:  59500
Query:  59600
Query:  59700
Query:  59800
Query:  59900
Query:  60000
Query:  60100
Query:  60200
Query:  60300
Query:  60400
Query:  60500
Query:  60600
Query:  60700
Query:  60800
Query:  60900
Query:  61000
Query:  61100
Query:  61200
Query:  61300
Query:  61400
Query:  61500
Query:  61600
Query:  61700
Query:  61800
Query:  61900
Query:  62000
Query:  62100
Query:  62200
Query:  62300
Query:  62400
Query:  62500
Query:  62600
Query:  62700
Query:  62800
Query:  62900
Query:  63000
Query:  63100
Query:  63200
Query:  63300
Query:  63400
Query:  63500
Query:  63600
Query:  63700
Query:  63800
Query:  63900
Query:  64000
Query:  64100
Query:  64200
Query:  64300
Query:  64400
Query:  64500
Query:  64600
Query:  64700
Query:  64800
Query:  64900
Query:  65000
Query:  65100
Query:  65200
Query:  65300
Query:  65400
Query:  65500
Query:  65600
Query:  65700
Query:  65800
Query:  65900
Query:  66000
Query:  66100
Query:  66200
Query:  66300
Query:  66400
Query:

Query:  116800
Query:  116900
Query:  117000
Query:  117100
Query:  117200
Query:  117300
Query:  117400
Query:  117500
Query:  117600
Query:  117700
Query:  117800
Query:  117900
Query:  118000
Query:  118100
Query:  118200
Query:  118300
Query:  118400
Query:  118500
Query:  118600
Query:  118700
Query:  118800
Query:  118900
Query:  119000
Query:  119100
Query:  119200
Query:  119300
Query:  119400
Query:  119500
Query:  119600
Query:  119700
Query:  119800
Query:  119900
Query:  120000
Query:  120100
Query:  120200
Query:  120300
Query:  120400
Query:  120500
Query:  120600
Query:  120700
Query:  120800
Query:  120900
Query:  121000
Query:  121100
Query:  121200
Query:  121300
Query:  121400
Query:  121500
Query:  121600
Query:  121700
Query:  121800
Query:  121900
Query:  122000
Query:  122100
Query:  122200
Query:  122300
Query:  122400
Query:  122500
Query:  122600
Query:  122700
Query:  122800
Query:  122900
Query:  123000
Query:  123100
Query:  123200
Query:  123300
Query:  12

In [35]:
# for k,v in alldfdata.items():
#     print(k, len(v))
print(fails)

24470


In [36]:
df = pd.concat(allpreds)

In [37]:
df.head(6)

,left,table,col,op,val,qid,dtype,dkind,likedtype,likekind,discrete_ops,in_ops,cont_ops,like_ops
0,tags,X,X,like,'%<adobe-illustrator>%',0,-1,-1,-1,contains,0.0,0.0,0.0,1.0
0,posts.ViewCount,posts,ViewCount,>,1000000,2,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,tcs.CusterStatus,tcs,CusterStatus,=,'Active',3,string,eq,-1,-1,1.0,0.0,0.0,0.0
0,ParentId,X,X,=,359903,7,num,eq,-1,-1,1.0,0.0,0.0,0.0
0,p.PostTypeId,p,PostTypeId,=,1,8,num,eq,-1,-1,1.0,0.0,0.0,0.0
1,Month(p.CreationDate),function,p.CreationDate,=,Month(getdate()),8,date,eq,-1,-1,1.0,0.0,0.0,0.0


In [38]:
print(df.keys())
print(len(df))

Index(['left', 'table', 'col', 'op', 'val', 'qid', 'dtype', 'dkind',
       'likedtype', 'likekind', 'discrete_ops', 'in_ops', 'cont_ops',
       'like_ops'],
      dtype='object')
412629


In [39]:
#df.to_csv("ceb_queries_df.csv", index=False)
#df.to_csv("tpcds_queries_df.csv", index=False)
df.to_csv("stackexchange_queries_df.csv", index=False)

In [40]:
# df2 = pd.read_csv("job_queries_df.csv")
# df2.head(5)

In [41]:
HTML(df[["like_ops", "discrete_ops", "cont_ops", "in_ops"]].\
     describe(percentiles=[0.9,0.99]).reset_index().to_html(index=False))

index,like_ops,discrete_ops,cont_ops,in_ops
count,391249.000000,391249.000000,391249.000000,391249.000000
mean,0.201741,0.610149,0.167124,0.034405
std,0.401300,0.487717,0.373087,0.182268
min,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000
90%,1.000000,1.000000,1.000000,0.000000
99%,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [42]:
#print(set(df["dkind"]))

In [43]:
discdf = df[df.discrete_ops == 1]
discdf.groupby(["dkind"]).count()["discrete_ops"].reset_index().sort_values(by="discrete_ops")

,dkind,discrete_ops
2,neq,1140
1,in,13461
0,eq,224119


In [44]:
discdf.groupby(["dtype"]).count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

,dtype,discrete_ops
3,string,134695
1,num,102370
2,sql,1461
0,date,194


In [45]:
ldf = df[df.like_ops == 1]
ldf.groupby("likedtype").count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

,likedtype,discrete_ops
6,unknown,62182
0,-1,10164
5,short,4577
4,serial,583
7,url,562
3,path,347
2,num,344
1,extension,172


In [75]:
ldf.groupby("likekind").count()["discrete_ops"].reset_index().sort_values(by="discrete_ops", ascending=False)

,likekind,discrete_ops
5,contains,28053
10,x%y%x,4189
7,no%,4024
6,ends,2715
8,starts,1646
1,3%,532
2,4%,79
9,x%x,72
3,5%,16
4,6%,7


In [76]:
inpdata = pd.DataFrame(allinpdata)

In [78]:
print(len(inpdata))
inpdata = inpdata.drop_duplicates()
inpdata = inpdata[inpdata["col"] != "function"]
print(len(inpdata))
print(len(set(inpdata["col"])))
print(set(inpdata["col"]))

151891
151891
483
{'', '@WithoutVotes', 'voterid', 'ReviewTaskTypeId', '@includeCactus', 'Owneruserid', 'owneruserid', 'AnswerCount', '@anum', '@UserCount', '[Name]', 'Accepted', '[Posttypeid]', '@user_id', '@total', 'Tags', '[Status]', 'p.CommentCount', 'DISPLAYNAME', 'title', '@rcnt', 'CommunityOwnedDate', 'b.Body', 'DateRank', 'is_ms_shipped', 'rnasc', '[TagBased]', 'Location', 'rk', 'VoteRevNo', 'revisionguid', 'PostHistoryTypeId', 'yyyy,CreationDate', 'OwnerUserID', 'object_id', 'CreationDate as char(10)', '@excludetag', 'b.Date', 'MigrationHistory.CreationDate', 'tagBased', 'C.Body', 'RN_A', 'fav', 'parentid', 'PostHistory.Comment AS int', 'BountyAmount', '@Empty', 'CreationDate AS DATE', 'Tagbased', 'P.AnswerCount', 'flagtypeid', 'ScoreNow', 'DATEDIFF(DAY, ClosedDate, GETDATE()) as FLOAT', 'CloseAsOffTopicReasonTypeId', '[Answers]', 'tagid', 'dd,CreationDate,LastAccessDate', 'p.CreationDate', 'TagBased', 'userId', 'Id', 'AuditTypeId', 'rnk', 'userid', 'Questions.CreationDate', '

# unique values seen in each column

In [80]:
inpdata.groupby("col").count().reset_index()["val"].describe(percentiles=[0.75,0.9,0.99])

count      483.000000
mean       314.474120
std       4399.790945
min          1.000000
50%          1.000000
75%          3.500000
90%         19.800000
99%       3715.440000
max      93337.000000
Name: val, dtype: float64

In [81]:
dff = df[df["col"] == "function"]
dff

,left,table,col,op,val,qid,dtype,dkind,likedtype,likekind,discrete_ops,in_ops,cont_ops,like_ops
3,(\n\t\t\tselect count(B.Id) \n\t\t\tfrom Posts...,function,function,>,0,66,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,N,X,function,>=,1,67,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,ParentId,X,function,=,135169,68,num,eq,-1,-1,1.0,0.0,0.0,0.0
0,Tags,X,function,like,'%google-cloud-functions%',71,-1,-1,serial,contains,0.0,0.0,0.0,1.0
0,PostTypeId,X,function,=,2,72,num,eq,-1,-1,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,(select sum(u3.Reputation) from Users u3 where...,function,function,>,1000,84765,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,PostHistoryTypeId,X,function,=,10,84766,num,eq,-1,-1,1.0,0.0,0.0,0.0
2,(up+down),function,function,>,0,84788,-1,-1,-1,-1,0.0,0.0,1.0,0.0
1,-2*down,function,function,<,0,84802,-1,-1,-1,-1,0.0,0.0,1.0,0.0


In [82]:
#pdf = pd.concat(allpreds)
pdf = df

In [83]:
pdf.head(5)

,left,table,col,op,val,qid,dtype,dkind,likedtype,likekind,discrete_ops,in_ops,cont_ops,like_ops
0,tcs.CusterStatus,tcs,CusterStatus,=,'Active',0,string,eq,-1,-1,1.0,0.0,0.0,0.0
0,posts.ViewCount,posts,ViewCount,>,10000,2,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,Posts.CreationDate,Posts,CreationDate,<,'2010-01-01',3,-1,-1,-1,-1,0.0,0.0,1.0,0.0
1,Posts.CreationDate,Posts,CreationDate,>,'2010-02-01',3,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,answer.PostTypeId,answer,PostTypeId,=,2,4,num,eq,-1,-1,1.0,0.0,0.0,0.0


In [84]:
#pdf = pdf[pdf.col.isin(COLS_TO_TABLES)]

# TODO: need equivalent version for SCOPE;
## TODO: more important is #columns per input in a query; for this we need column --> table mapping

In [85]:
pdf.groupby(["col", "qid"])["op"].nunique().reset_index().sort_values(by="op", ascending=False)["op"].describe(
                                    percentiles=[0.75,0.90,0.99])

count    76202.000000
mean         1.156964
std          0.421617
min          1.000000
50%          1.000000
75%          1.000000
90%          2.000000
99%          3.000000
max          6.000000
Name: op, dtype: float64

In [86]:
for col in set(pdf["col"]):
    if not col in COLS_TO_TABLES:
        print(col)

nan

answer.Body
 reputation
m,CreationDate
DD,p.CreationDate
Users.LastAccessDate
ReviewTaskTypeId
m,q.creationdate
p.Id
owneruserid
description
c
AnswerCount
pp.Body
Rep
@anum
[Name]
dd,Question.CreationDate,Answer.CreationDate
ss,Users.CreationDate,Posts.CreationDate
ReviewResultType
[u].[Location]
V.CreationDate
P.Body
[Posttypeid]
q.Tags
Tags
[Status]
s,creationdate,closeddate
p.CommentCount
Monthindex
title
Answers.Body
q.Title
p.CreationDate,@latestDate
CommunityOwnedDate
b.Title
[Accepted]
b.Body
d,LastAccessDate
##TagName
p.Title
is_ms_shipped
Location
num
Users.DisplayName
[Order]
mi,p7.CreationDate,p6.CreationDate
VoteRevNo
Questions.CreationDate,Votes.CreationDate
phis.Comment
PostHistoryTypeId
us.DisplayName
v.CreationDate,p.CreationDate
yyyy,CreationDate
dd,users.CreationDate
index_id
dd,LastAccessDate
a.body
p.CreationDate,@EndDate
Posts.CreationDate,@latest
d,CreationDate
OwnerUserID
object_id
dd,CreationDate,LastPostedDate
CreationDate as char(10)
b.Date
MigrationHisto

In [87]:
pdf2 = pdf[pdf.table != "X"]
print(set(pdf2["table"]))
print(len(set(pdf2["table"])))

{nan, 'vup', 'uc', 'pAll', 'csh', 'c', 'Results', 'mdi', 'subqAllAnswers', 'Ups', 'o1', 'qtn', 'CSH', 'me', 'v', 'A2', 'Tags', 'AnswerPosts', 'counts', 'TagCount', 'hp', 'TotalAnswers', '[#CustomerStatusHistory]', 'A', 'uu', 'num', 'PostHistory', '[ph]', 'postsq', 'ya', 'v2', 'AT', 'AcceptedAnswer', 'pall', 'e', 'ph0', 'u3', 'p1', 'qc', 'PostsA', 'tag1', 'pg', 'answer', 'a2', 'temp', 't', 'W', 'pa2', 'se', 'QUESTIONS', 'CloseSide', 'function', 'queries', 'f', 'wdv', 'uv', 'Bins', 'qbt', 'wk', 'accepted', 'actor', 'yourAnswer', 'AcceptVote', 'ht', 'y', 'ReviewTaskTypes', 'Y', 'naa', 'pt', 'bt', 'PA', 'k', 'answ', '[p]', 'tags', 'N', 'reply', 'fu', 'USERS', 'DS', 's1', 'vt', 'ContQ', 'Del', 'UserQuestions', 'Pht', 'hello', 'PF', 'ptypes', 'pans', 'prev', 'PythonBadges', 'votetypes', 'su', 'ReopenSide', 'ANSWERS', 'V', 'B', 'allTheTags', 'pc', 'Pt', 'comment', 'tagsY', 'pm', 'data0', 'this', 'v0', 'tagsX', 'status_tag', 'rollbacks', 'E', 'p_a', 'Raw', 'AcceptedAnswers', 'i2', '[se]', 'bty

In [88]:
pdf2.groupby(["table", "qid"])["col"].nunique().reset_index().sort_values(by="col", ascending=False)["col"].describe(
                                    percentiles=[0.75,0.90,0.99])

count    48325.000000
mean         1.207926
std          0.509182
min          1.000000
50%          1.000000
75%          1.000000
90%          2.000000
99%          3.000000
max          8.000000
Name: col, dtype: float64

In [89]:
pd.set_option('display.max_rows', 500)
pdf2.head(100)

,left,table,col,op,val,qid,dtype,dkind,likedtype,likekind,discrete_ops,in_ops,cont_ops,like_ops
0,tcs.CusterStatus,tcs,CusterStatus,=,'Active',0,string,eq,-1,-1,1.0,0.0,0.0,0.0
0,posts.ViewCount,posts,ViewCount,>,10000,2,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,Posts.CreationDate,Posts,CreationDate,<,'2010-01-01',3,-1,-1,-1,-1,0.0,0.0,1.0,0.0
1,Posts.CreationDate,Posts,CreationDate,>,'2010-02-01',3,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,answer.PostTypeId,answer,PostTypeId,=,2,4,num,eq,-1,-1,1.0,0.0,0.0,0.0
1,question.Score,question,Score,<,25,4,-1,-1,-1,-1,0.0,0.0,1.0,0.0
0,Votes.VoteTypeId,Votes,VoteTypeId,=,9,6,num,eq,-1,-1,1.0,0.0,0.0,0.0
0,Tags.TagName,Tags,TagName,=,'##Tag##',7,string,eq,-1,-1,1.0,0.0,0.0,0.0
4,A.CreationDate,A,OwnerUserId,>,'2016-01-01',7,num,in,-1,-1,1.0,1.0,0.0,0.0
5,A.OwnerUserId,NaN,NaN,in,"(\n 209103, -- puf\n 394010, -- katowulf...",7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
pdf2.groupby(["table", "qid"])["col"].nunique().reset_index().sort_values(by="col", ascending=False)

,table,qid,col
23073,function,30456,8
40790,q,57705,6
40198,q,32748,6
41359,q,81875,6
39920,q,21505,6
...,...,...,...
18007,d,25890,1
18008,d,26714,1
18009,d,30783,1
18011,d,39438,1


# TODO:

#### More columns being filtered (tails) on BUT within a column ---- fewer categorical values being hit
#### discrete_type: num vs string
#### cont dtypes: range vs leq vs geq

In [91]:
pdf.groupby("op").count()["qid"]

op
!=            568
<            7646
<=           2780
<>           1214
=           93323
>           14509
>=           6438
between      1218
in           9267
like        41662
not in        517
not like     4972
Name: qid, dtype: int64